In [ ]:
import numpy as np
import cv2
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image, ImageDraw 
import argparse
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import os
import matplotlib.pyplot as plt
import time
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
from torchvision import models
class Net_res(torch.nn.Module):
    def __init__(self):
        '''
        初始化一些层，使用resnet18作为backbone
        backbone最后一层输出改变成18
        '''
        super(Net_res,self).__init__()
        self.resnet18 = models.resnet18(pretrained= True)
        self.resnet18.fc = torch.nn.Linear(in_features=512, out_features= 256, bias= True)
        self.last_fc = torch.nn.Linear(256, 18, bias= True)
        self.prelu = torch.nn.PReLU()
    
    def forward(self, x):
        '''
        传播过程
        '''
        #输入尺寸为224*224
        x = self.resnet18(x)
        x = self.prelu(x)
        x = self.last_fc(x)
        return x

In [ ]:
class Normalize(object):
    """
        Resieze to train_boarder x train_boarder. Here we use 224 x 224
        Then do channel normalization: (image - mean) / std_variation
    """
    def __init__(self,train_boarder= 224):
        self.train_boarder = train_boarder
    def __call__(self,sample):
        img, w, h = sample['image'], sample['w'], sample['h']
        img_resize = np.asarray(img.resize((self.train_boarder, self.train_boarder),Image.BILINEAR))
        return {'image':img_resize, 'image_ori':np.asarray(img), 'w' : w, 'h' : h}

In [ ]:
class ToTensor(object):
    """
        Convert ndarrays in sample to Tensors.
        Tensors channel sequence: N x C x H x W
    """
    def __call__(self,sample):
        '''
        numpy img: H*W*C
        torch.tensorimg: N*C*H*W
        '''
        img, w, h = sample['image'], sample['w'], sample['h']
#         如果不是灰度图要改变维度
        img_transposed = img.transpose((2, 0, 1))
        return {'image':torch.from_numpy(img_transposed).float(), 'image_ori':sample['image_ori'], 'w' : w, 'h' : h}

In [ ]:
class FaceLandmarksDataset():
    def __init__(self, data, transforms= None, train_boarder= 224):
        '''
        :param lines: src_line
        :param transform: data transform
        '''
        self.data = data
        self.transforms = transforms
        self.train_boarder = train_boarder
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img = self.data.values[idx][0]
        #打开图片
        img = Image.open(img)
        
        #对lanmarks做变换
        w = img.width
        h = img.height
        
        sample = {'image':img, 'w' : w, 'h' : h}
    
        sample = self.transforms(sample)
        return sample

In [ ]:
def load_data(filepath):
    '''
    加载数据
    '''
    df = pd.read_csv(filepath)
    tsfm = transforms.Compose([
        Normalize(),
        ToTensor()]
    )
    data_set = FaceLandmarksDataset(df, transforms= tsfm)
    return data_set


In [ ]:
def main():
    t1 = time.time()
    path = 'data/test.csv'
    predict_set = load_data(path)
    predict_loader = torch.utils.data.DataLoader(predict_set, 
                                             batch_size= 1, 
                                             num_workers= 1, 
                                             pin_memory= False)
    path_model = 'trained_models/detector_epoch_100.pt'
    device = 'cuda'
    model = Net_res().to(device)
    model.load_state_dict(torch.load(path_model, map_location= device))
    model.eval()
    t2 = time.time()
    landmarks = []
    with torch.no_grad():
        for i, data in enumerate(predict_loader):
            if i % 100 == 0:
                print('\r','正在处理第{}张猫片'.format(i),end= '')
            img = data['image'].to(device)
            w = data['w'].item()
            h = data['h'].item()
            output_pts = model(img)
            output_pts = output_pts[0].cpu().numpy()
            output_pts[::2] = output_pts[::2]/224*w
            output_pts[1::2] = output_pts[1::2]/224*h
            landmarks.append(output_pts)
    t3 = time.time()
    print(t2 - t1)
    print(t3 - t2)
    return np.array(landmarks)

In [ ]:
if __name__ == '__main__':
    landmarks = main()

In [ ]:
pd.DataFrame(landmarks).to_csv('data/results.csv')

In [ ]:
t1 = time.time()
path = 'data/test.csv'
predict_set = load_data(path)
t2 = time.time()
print('load_data:', t2 - t1)
predict_loader = torch.utils.data.DataLoader(predict_set, 
                                             batch_size= 1, 
                                             num_workers= 1, 
                                             pin_memory= False)
t3 = time.time()
print('data_loader:', t3 - t2)
path_model = 'trained_models/detector_epoch_100.pt'
device = 'cuda'
model = Net_res().to(device)
t4 = time.time()
print('to_device:', t4 - t3)
model.load_state_dict(torch.load(path_model, map_location= device))
model.eval()
idx = 25
t5 = time.time()
print('load_model:', t5 - t4)
landmarks = []
with torch.no_grad():
    for i,data in enumerate(predict_loader):
        if i == idx:
            img = data['image'].to(device)
            w = data['w'].item()
            h = data['h'].item()
            img_ori = data['image_ori']
            output_pts = model(img)
            landmarks = output_pts[0].cpu().numpy()
            
            xs = landmarks[::2]/224*w
            ys = landmarks[1::2]/224*h
            plt.imshow(img_ori[0].numpy())
            plt.scatter(xs,ys,s = 5, c = 'r', edgecolors= 'r')
            plt.show()
        elif i > idx:
            break
t6 = time.time()
print('run:', t6 - t5)

In [ ]:
t1 = time.time()
path = 'data/test.csv'
predict_set = load_data(path)
t2 = time.time()
print('load_data:', t2 - t1)
predict_loader = torch.utils.data.DataLoader(predict_set, 
                                             batch_size= 1, 
                                             num_workers= 0, 
                                             pin_memory= False)
t3 = time.time()
print('data_loader:', t3 - t2)
path_model = 'trained_models/detector_epoch_100.pt'
device = 'cuda'
model = Net_res().to(device)
t4 = time.time()
print('to_device:', t4 - t3)
model.load_state_dict(torch.load(path_model, map_location= device))
model.eval()
idx = 22
t5 = time.time()
print('load_model:', t5 - t4)
landmarks = []
with torch.no_grad():
    for i,data in enumerate(predict_loader):
        if i == idx:
            img = data['image'].to(device)
            w = data['w'].item()
            h = data['h'].item()
            img_ori = data['image_ori']
            output_pts = model(img)
            landmarks = output_pts[0].cpu().numpy()
            
            xs = landmarks[::2]/224*w
            ys = landmarks[1::2]/224*h
            plt.imshow(img_ori[0].numpy())
            plt.scatter(xs,ys,s = 5, c = 'r', edgecolors= 'r')
            plt.show()
        elif i > idx:
            break
t6 = time.time()
print('run:', t6 - t5)

In [ ]:
img = Image.open('data/test/0.jpg')

img_resize = np.asarray(img.resize((224, 224),Image.BILINEAR))

img_tensor = torch.from_numpy(img_resize.transpose((2, 0, 1))).float()

path_model = 'trained_models_Copy2/detector_epoch_100.pt'
model = Net_res()
device = 'cpu'
model.load_state_dict(torch.load(path_model, map_location= device))
model.eval()
with torch.no_grad():
    out = model(img_tensor.unsqueeze(0))

xs = out[0,::2]
ys = out[0,1::2]

plt.imshow(img_resize)
plt.scatter(xs,ys,s = 5, c = 'r', edgecolors= 'r')
plt.show()